https://github.com/miguelfzafra/Latest-News-Classifier/blob/master/0.%20Latest%20News%20Classifier/03.%20Feature%20Engineering/03.%20Feature%20Engineering.ipynb

https://github.com/miguelfzafra/Latest-News-Classifier/blob/master/0.%20Latest%20News%20Classifier/04.%20Model%20Training/06.%20MT%20-%20Random%20Forest.ipynb

https://towardsdatascience.com/text-classification-in-python-dd95d264c802

https://github.com/miguelfzafra/Latest-News-Classifier/tree/master/0.%20Latest%20News%20Classifier/04.%20Model%20Training



1. Read in the data
2. Test train split
3. use TFIFD to add in the new features
4. Use an algorithm to make some predictions
5. Assess the accuracy

In [1]:
DATA_IN_FOLDER : str = "data/out/"
    
SOURCE_DATA_FILE : str = "Womens Clothing E-Commerce Reviews with Emotions.xlsx"    

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2

In [3]:
df_reviews = pd.read_excel(f"{DATA_IN_FOLDER}{SOURCE_DATA_FILE}")
df_reviews.dropna(inplace=True) 
df_reviews.reset_index(drop=True, inplace=True)
df_reviews.head()

,index,Review Title,Review Text,Review Rating,Review Positive Feedback Count,Review Polarity,Review Sentiment,Review Subjectivity,Review Length,Review Word Count,...,Reviewer Age,Reviewer Age Category,Emotion,Emotion Score,Division,Department,Product ID,Product Name,Product Category,Recommended?
0,1,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,0,0.550000,Positive,0.250000,124,22,...,50,45-54,Joy,0.972142,General Petite,Bottoms,1049,Product 1049,Pants,1
1,2,Flattering shirt,This shirt is very flattering to all due to th...,5,6,0.512891,Positive,0.137500,192,36,...,47,45-54,Joy,0.844207,General,Tops,847,Product 847,Blouses,1
2,3,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,4,0.178750,Positive,0.066250,488,98,...,49,45-54,Joy,0.668897,General,Dresses,1080,Product 1080,Dresses,0
3,4,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,0.133750,Positive,0.215556,496,101,...,39,35-44,Joy,0.573683,General Petite,Tops,858,Product 858,Knits,1
4,5,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,4,0.171635,Positive,-0.007692,482,97,...,39,35-44,Joy,0.554208,General Petite,Tops,858,Product 858,Knits,1


In [4]:
df_reviews['Emotion'].value_counts()

Joy        1588
Sadness     137
Anger        15
Fear         13
Name: Emotion, dtype: int64

In [5]:
emotion_map = {
    'Joy': 0,
    'Sadness': 1,
    'Fear': 2,
    'Anger': 3
}

df_reviews['Emotion Code'] = df_reviews['Emotion']
df_reviews = df_reviews.replace({'Emotion Code': emotion_map})
df_reviews.head()

,index,Review Title,Review Text,Review Rating,Review Positive Feedback Count,Review Polarity,Review Sentiment,Review Subjectivity,Review Length,Review Word Count,...,Reviewer Age Category,Emotion,Emotion Score,Division,Department,Product ID,Product Name,Product Category,Recommended?,Emotion Code
0,1,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,0,0.550000,Positive,0.250000,124,22,...,45-54,Joy,0.972142,General Petite,Bottoms,1049,Product 1049,Pants,1,0
1,2,Flattering shirt,This shirt is very flattering to all due to th...,5,6,0.512891,Positive,0.137500,192,36,...,45-54,Joy,0.844207,General,Tops,847,Product 847,Blouses,1,0
2,3,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,4,0.178750,Positive,0.066250,488,98,...,45-54,Joy,0.668897,General,Dresses,1080,Product 1080,Dresses,0,0
3,4,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,0.133750,Positive,0.215556,496,101,...,35-44,Joy,0.573683,General Petite,Tops,858,Product 858,Knits,1,0
4,5,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,4,0.171635,Positive,-0.007692,482,97,...,35-44,Joy,0.554208,General Petite,Tops,858,Product 858,Knits,1,0


In [6]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1753 entries, 0 to 1752
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           1753 non-null   int64  
 1   Review Title                    1753 non-null   object 
 2   Review Text                     1753 non-null   object 
 3   Review Rating                   1753 non-null   int64  
 4   Review Positive Feedback Count  1753 non-null   int64  
 5   Review Polarity                 1753 non-null   float64
 6   Review Sentiment                1753 non-null   object 
 7   Review Subjectivity             1753 non-null   float64
 8   Review Length                   1753 non-null   int64  
 9   Review Word Count               1753 non-null   int64  
 10  Review Text Cleaned             1753 non-null   object 
 11  Review Text Wordcloud           1753 non-null   object 
 12  Reviewer Age                    17

In [7]:
# Upsample the reviews
n = df_reviews.shape[0] // len(df_reviews['Emotion Code'].unique())
df_reviews = df_reviews.groupby("Emotion Code").sample(n=n, random_state=42, replace=True)
df_reviews.reset_index(drop=True, inplace=True)

In [8]:
df_reviews['Emotion Code'].value_counts()

0    438
1    438
2    438
3    438
Name: Emotion Code, dtype: int64

In [9]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8', ngram_range=ngram_range, stop_words=None, lowercase=False, max_df=max_df, min_df=min_df, max_features=max_features, norm='l2', sublinear_tf=True)

X = df_reviews['Review Text Wordcloud']
y = df_reviews['Emotion Code']

tfid_features = tfidf_vectorizer.fit_transform(X).toarray()


In [10]:
def get_correlated_ngrams(emotion_code):

    features_chi2 = chi2(tfid_features, y == emotion_code) # For the current emotion, use chi2 stats to evaluate the highest correlations with the target
    indices = np.argsort(features_chi2[0])[::-1] # np.argsort returns the indices that would sort the array ascending, [::-1] makes it descending with the highest correlation in first place
    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices] # get_feature_names uses the sorted indices to order the features most to least correlated with the target
    
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1] # Selects the feature names that are single words
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2] # Selects the feature names that are two words
    
    return unigrams, bigrams

In [11]:
for emotion, emotion_code in sorted(emotion_map.items()):
    unigrams, bigrams = get_correlated_ngrams(emotion_code)
    
    print(f"{emotion}:")
    print(f"Most correleated unigrams: {', '.join(unigrams[:20])}")
    print(f"Most correleated bigrams: {', '.join(bigrams[:5])}\n")

Anger:
Most correleated unigrams: problem, purchased, pounds, tank, dressy, strike, idea, hanger, flat, open, underneath, appears, exposed, seems, shows, deep, gorgeous, red, quite, reviews
Most correleated bigrams: need wear, pounds usually, reviewers mentioned, size small, runs small

Fear:
Most correleated unigrams: fear, underlayer, pulled, slightly, across, big, squareapple, paint, types, si, space, done, narrow, romper, barely, aline, shoulders, puffy, vest, dots
Most correleated bigrams: runs slightly, types fabric, puffy paint, would say, top looks

Joy:
Most correleated unigrams: perfect, got, great, fits, nice, love, pounds, pair, true, jeans, bad, wearing, design, many, chest, warm, fear, vneck, lay, dry
Most correleated bigrams: true size, runs slightly, need wear, dry clean, fabric bad

Sadness:
Most correleated unigrams: disappointed, sad, suit, saw, bottom, wore, get, pounds, received, stiff, first, great, love, perfect, see, lay, regular, wanted, like, elastic
Most corr

Now with 2000 calls into the IBM Watson API and by sampling the data we have a plausible looking set of top unigrams to feed into our own customised algorithm!!

So, ... we can now try joining the tfid_features onto the main data, splitting into train and test and building an algorithm ...

In [12]:
pd.DataFrame(tfid_features)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.355607,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.294135,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.198444,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1747,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
1748,0.0,0.0,0.256952,0.0,0.0,0.000000,0.0,0.0,0.227067,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.497203
1749,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
1750,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.216609,0.226117,0.0,0.0,0.0,0.000000


In [13]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752 entries, 0 to 1751
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           1752 non-null   int64  
 1   Review Title                    1752 non-null   object 
 2   Review Text                     1752 non-null   object 
 3   Review Rating                   1752 non-null   int64  
 4   Review Positive Feedback Count  1752 non-null   int64  
 5   Review Polarity                 1752 non-null   float64
 6   Review Sentiment                1752 non-null   object 
 7   Review Subjectivity             1752 non-null   float64
 8   Review Length                   1752 non-null   int64  
 9   Review Word Count               1752 non-null   int64  
 10  Review Text Cleaned             1752 non-null   object 
 11  Review Text Wordcloud           1752 non-null   object 
 12  Reviewer Age                    17

In [14]:
X = pd.concat([df_reviews.iloc[:,[3,5]], pd.DataFrame(tfid_features)], axis=1)
y = df_reviews['Emotion Code']

There is a bug in pandas version 1.1.3 which will cause ``.info(verbose=True)`` to crash where the DataFrame contains column names that are integers (see https://github.com/pandas-dev/pandas/issues/37408).

The solution is to upgrade to the latest version -

```
pip install --user --upgrade pandas
```

In [15]:
pd.__version__

'1.3.5'

In [16]:
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752 entries, 0 to 1751
Data columns (total 302 columns):
 #    Column           Dtype  
---   ------           -----  
 0    Review Rating    int64  
 1    Review Polarity  float64
 2    0                float64
 3    1                float64
 4    2                float64
 5    3                float64
 6    4                float64
 7    5                float64
 8    6                float64
 9    7                float64
 10   8                float64
 11   9                float64
 12   10               float64
 13   11               float64
 14   12               float64
 15   13               float64
 16   14               float64
 17   15               float64
 18   16               float64
 19   17               float64
 20   18               float64
 21   19               float64
 22   20               float64
 23   21               float64
 24   22               float64
 25   23               float64
 26   24               f

In [17]:
X.head()

,Review Rating,Review Polarity,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,5,0.442045,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.355607,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.275000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.294135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.225000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.406052,0.0,0.0,0.0,0.0,0.0,0.198444,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.385714,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
y.value_counts()

0    438
1    438
2    438
3    438
Name: Emotion Code, dtype: int64